In [1]:
import glob, os
import pandas as pd
import xlrd

In [2]:
path = 'C:/Users/WB459082/Documents/Andres/RATES'
sheet_names = ['spot', 'MSC 21days', 'COSCO', 'ONE', 'HPL spot', 'HBS']
os.listdir(path)

['.ipynb_checkpoints',
 'appended_complete.xlsx',
 'appended_complete_clean.xlsx',
 'Book1.twb',
 'Book2.twb',
 'do_file.do',
 'fwdfyirates.zip',
 'image003.jpg',
 'images',
 'MEXICO RATE-als-update 2018.xls',
 'MEXICO RATE-als-update 2018_1.xls',
 'MEXICO RATE-als-update 2018_2.xls',
 'MEXICO RATE-als-update 2018_3.xls',
 'MEXICO RATE-als-update 2018_4.xls',
 'msc21days.xlsx',
 'Rates Database.ipynb',
 'spot.xlsx',
 'Validity Dates.xlsx']

In [3]:
# Replace a set of multiple sub strings with a new string in main string.

def replaceMultiple(mainString, toBeReplaces, newString):
    # Iterate over the strings to be replaced
    for elem in toBeReplaces :
        # Check if string is in the main string
        if elem in mainString :
            # Replace the string
            mainString = mainString.replace(elem, newString)
    
    return  mainString

# Validity dates

In [4]:
remove = ['2018', '2019', 'th']
head = "File\tSheet\tStart date\tEnd date"
print(head)
for sheet in sheet_names:
    row = 2
    if sheet == 'spot':
        row=12
    for file in os.listdir(path):
        if file.endswith(".xls"):
            xl_workbook = xlrd.open_workbook(file)
            xl_sheet = xl_workbook.sheet_by_name(sheet)
            start = xl_sheet.row(row)[0].value.split(" to ")[0].split("from ",1)[1].replace('th','').replace(',',' ').replace('2018','').replace('2019','')
            end = xl_sheet.row(row)[0].value.split(" to ")[1].split("by",1)[0].replace('th','').replace(',',' ').replace('2018','').replace('2019','')
            start2 = replaceMultiple(xl_sheet.row(row)[0].value.split(" to ")[0].split("by",1)[0],remove, "").replace(',',' ')
            end2 = replaceMultiple(xl_sheet.row(row)[0].value.split(" to ")[1].split("by",1)[0],remove, "").replace(',',' ')
            print(file,sheet,start + " 2019",end + " 2019",sep="\t \t")


File	Sheet	Start date	End date
MEXICO RATE-als-update 2018.xls	 	spot	 	25 Nov 2019	 	30 Nov  2019
MEXICO RATE-als-update 2018_1.xls	 	spot	 	18 Nov 2019	 	24 Nov  2019
MEXICO RATE-als-update 2018_2.xls	 	spot	 	11 Nov 2019	 	24 Nov  2019
MEXICO RATE-als-update 2018_3.xls	 	spot	 	28 Oct 2019	 	10 Nov  2019
MEXICO RATE-als-update 2018_4.xls	 	spot	 	23 Sep 2019	 	6 Oct  2019
MEXICO RATE-als-update 2018.xls	 	MSC 21days	 	25 Nov 2019	 	30 Nov 2019
MEXICO RATE-als-update 2018_1.xls	 	MSC 21days	 	18 Nov 2019	 	24 Nov 2019
MEXICO RATE-als-update 2018_2.xls	 	MSC 21days	 	11 Nov 2019	 	24 Nov 2019
MEXICO RATE-als-update 2018_3.xls	 	MSC 21days	 	28 Oct 2019	 	10 Nov 2019
MEXICO RATE-als-update 2018_4.xls	 	MSC 21days	 	23 Sep 2019	 	6 Oct 2019
MEXICO RATE-als-update 2018.xls	 	COSCO	 	25 Nov 2019	 	30 Nov 2019
MEXICO RATE-als-update 2018_1.xls	 	COSCO	 	15 Nov 2019	 	24 Nov 2019
MEXICO RATE-als-update 2018_2.xls	 	COSCO	 	11 Nov 2019	 	24 Nov 2019
MEXICO RATE-als-update 2018_3.xls	 	COSCO	

In [5]:
metadata = pd.read_excel('Validity Dates.xlsx', index_col=0)

# Import all

In [6]:
completeList = []
for sheet in sheet_names:
    for file in os.listdir(path):
        if file.endswith(".xls"):
            xls_file = pd.ExcelFile(file)
            data = xls_file.parse(sheet,skiprows=3)
            data['File'] = file
            data['Sheet'] = sheet
            completeList.append(data) 
appended_complete = pd.concat(completeList,sort=True)
#appended_complete = appended_complete[appended_complete.iloc[:,1].notnull()]
appended_complete = appended_complete.merge(metadata, left_on=['File','Sheet'], right_on=['File','Sheet'])
appended_complete.to_excel(os.path.join(path, 'appended_complete.xlsx'))

In [8]:
appended_complete.columns

Index(['20'GP', '20GP', '40'GP', '40'HC', '40'Nor', '40GP', '40HQ', '40NOR',
       'AMS C/F', 'CY CLS', 'Demurrage', 'ETD', 'File', 'LOCATION',
       'M.CITY R+T', 'OWS', 'POL', 'POL/POD', 'REMARK', 'Sheet', 'T/T',
       'Transmode', 'Description', 'Start date', 'End date'],
      dtype='object')

In [9]:
appended_complete['Sheet'].value_counts()

ONE           530
MSC 21days    450
HPL spot      370
HBS           340
COSCO         230
spot           75
Name: Sheet, dtype: int64

In [10]:
appended_complete['File'].value_counts()

MEXICO RATE-als-update 2018_2.xls    399
MEXICO RATE-als-update 2018_3.xls    399
MEXICO RATE-als-update 2018_4.xls    399
MEXICO RATE-als-update 2018.xls      399
MEXICO RATE-als-update 2018_1.xls    399
Name: File, dtype: int64

In [11]:
appended_complete.describe(include='all')

,20'GP,20GP,40'GP,40'HC,40'Nor,40GP,40HQ,40NOR,AMS C/F,CY CLS,...,OWS,POL,POL/POD,REMARK,Sheet,T/T,Transmode,Description,Start date,End date
count,24.000000,1700,24.000000,24.000000,0.0,1685.0,1695.0,115.0,335,335,...,65,880,41,0.0,1995,620.0,560,525,1995,1995
unique,NaN,273,NaN,NaN,NaN,274.0,266.0,11.0,17,30,...,1,141,17,NaN,6,25.0,12,2,9,7
top,NaN,/,NaN,NaN,NaN,3200.0,3200.0,2500.0,/,/,...,NO,IHI ADD ON:,21 calendar days free line detention,NaN,ONE,26.0,VIA HKG,MSC +port charges usd10/20+lss usd20/40 includ...,2019-11-11 00:00:00,2019-11-24 00:00:00
freq,NaN,90,NaN,NaN,NaN,101.0,101.0,22.0,100,40,...,65,15,5,NaN,530,95.0,175,450,641,940
first,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-07-01 00:00:00,2019-07-07 00:00:00
last,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-11-25 00:00:00,2019-11-30 00:00:00
mean,2323.958333,NaN,2389.583333,2389.583333,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,524.895392,NaN,518.982945,518.982945,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,1300.000000,NaN,1300.000000,1300.000000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,1993.750000,NaN,2075.000000,2075.000000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Import spot 

In [237]:
sheet = 'spot'
spotList = []
for file in os.listdir(path):
    if file.endswith(".xls"):
        xls_file = pd.ExcelFile(file)
        data = xls_file.parse(sheet,skiprows=3)
        data['file'] = file
        data['sheet'] = sheet
        spotList.append(data) 
appended_spot = pd.concat(spotList,sort=True)
appended_spot = appended_spot[appended_spot.iloc[:,1].notnull()]
appended_spot.to_excel(os.path.join(path, 'appended_spot.xlsx'))


# Import MSC 21days

In [238]:
sheet = 'MSC 21days'
msc21daysList = []
for file in os.listdir(path):
    if file.endswith(".xls"):
        xls_file = pd.ExcelFile(file)
        data = xls_file.parse(sheet,skiprows=3)
        data['file'] = file
        data['sheet'] = sheet
        msc21daysList.append(data) 
appended_msc21days = pd.concat(msc21daysList,sort=True)
appended_msc21days = appended_msc21days[appended_msc21days.iloc[:,1].notnull()]
appended_msc21days.to_excel(os.path.join(path, 'appended_msc21days.xlsx'))

# Import COSCO

In [239]:
sheet = 'COSCO'
coscoList = []
for file in os.listdir(path):
    if file.endswith(".xls"):
        xls_file = pd.ExcelFile(file)
        data = xls_file.parse(sheet,skiprows=3)
        data['file'] = file
        data['sheet'] = sheet
        coscoList.append(data) 
appended_cosco = pd.concat(coscoList,sort=True)
appended_cosco = appended_cosco[appended_cosco.iloc[:,1].notnull()]
appended_cosco.to_excel(os.path.join(path, 'appended_cosco.xlsx'))

# Import ONE

In [240]:
sheet = 'ONE'
oneList = []
for file in os.listdir(path):
    if file.endswith(".xls"):
        xls_file = pd.ExcelFile(file)
        data = xls_file.parse(sheet,skiprows=3)
        data['file'] = file
        data['sheet'] = sheet
        oneList.append(data) 
appended_one = pd.concat(oneList,sort=True)
appended_one = appended_one[appended_one.iloc[:,1].notnull()]
appended_one.to_excel(os.path.join(path, 'appended_one.xlsx'))

# Import HPL Spot

In [241]:
sheet = 'HPL spot'
hplspotList = []
for file in os.listdir(path):
    if file.endswith(".xls"):
        xls_file = pd.ExcelFile(file)
        data = xls_file.parse(sheet,skiprows=3)
        data['file'] = file
        data['sheet'] = sheet
        hplspotList.append(data) 
appended_hplspot = pd.concat(hplspotList,sort=True)
appended_hplspot = appended_hplspot[appended_hplspot.iloc[:,1].notnull()]
appended_hplspot.to_excel(os.path.join(path, 'appended_hplspot.xlsx'))

# Import HBS

In [242]:
sheet = 'HBS'
hbsList = []
for file in os.listdir(path):
    if file.endswith(".xls"):
        xls_file = pd.ExcelFile(file)
        data = xls_file.parse(sheet,skiprows=3)
        data['file'] = file
        data['sheet'] = sheet
        hbsList.append(data) 
appended_hbs = pd.concat(hbsList,sort=True)
appended_hbs = appended_hbs[appended_hbs.iloc[:,1].notnull()]
appended_hbs.to_excel(os.path.join(path, 'appended_hbs.xlsx'))